# Ordinary differential equation model using ModelingToolkit
Simon Frost (@sdwfrost), 2020-05-04

## Introduction

The classical ODE version of the SIR model is:

- Deterministic
- Continuous in time
- Continuous in state

This version, unlike the 'vanilla' ODE version, uses [ModelingToolkit](https://mtk.sciml.ai/). For small problems such as this, it doesn't make much of a difference for compute time, but it is a little more expressive and lends itself to extending a little better.

## Libraries

In [ ]:
using DifferentialEquations
using ModelingToolkit
using OrdinaryDiffEq
using DataFrames
using StatsPlots
using BenchmarkTools

## Transitions

In [ ]:
@parameters t β c γ
@variables S(t) I(t) R(t)
D = Differential(t)
N=S+I+R # This is recognized as a derived variable
eqs = [D(S) ~ -β*c*I/N*S,
       D(I) ~ β*c*I/N*S-γ*I,
       D(R) ~ γ*I];

In [ ]:
@named sys = ODESystem(eqs);

## Time domain

We set the timespan for simulations, `tspan`.

In [ ]:
δt = 0.1
tmax = 40.0
tspan = (0.0,tmax)
t = 0.0:δt:tmax;

## Initial conditions

In `ModelingToolkit`, the initial values are defined by a dictionary.

In [ ]:
u0 = [S => 990.0,
      I => 10.0,
      R => 0.0];

## Parameter values

Similarly, the parameter values are also defined by a dictionary.

In [ ]:
p = [β=>0.05,
     c=>10.0,
     γ=>0.25];

## Running the model

In [ ]:
prob = ODEProblem(sys, u0, tspan, p; jac=true);

If we use `DifferentialEquations.jl`, we don't need to specify a solver, the package will choose it for us.

In [ ]:
sol = solve(prob);

We can check which solver was chosen as follows.

In [ ]:
sol.alg

## Post-processing

We can convert the output to a dataframe for convenience.

In [ ]:
df = DataFrame(sol(t))
rename!(df, [:t, :S, :I, :R]);

## Plotting

We can now plot the results.

In [ ]:
@df df plot(:t,
    [:S :I :R],
    xlabel="Time",
    ylabel="Number")

## Benchmarking

In [ ]:
@benchmark solve(prob)